In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from tensorflow.keras.optimizers import Adam

# 讀取CSV文件
df = pd.read_csv('/content/drive/MyDrive/archive/dogs.csv')

# 選擇指定類別
selected_breeds = [
    "Airedale", "Beagle", "Bloodhound", "Bluetick", "Chihuahua",
    "Collie", "Dingo", "French Bulldog", "German Sheperd",
    "Malinois", "Newfoundland", "Pekinese", "Pomeranian", "Pug", "Vizsla"
]

# 過濾指定的品種
df_selected = df[df['labels'].isin(selected_breeds)]

# 加載和預處理圖像
def load_images(df):
    image_list = []
    for idx, row in df.iterrows():
        img = Image.open('/content/drive/MyDrive/archive/' + row['filepaths'])
        img = img.resize((224, 224))  # 假設使用224x224大小的圖像
        img_array = np.array(img) / 255.0  # 圖像歸一化
        image_list.append(img_array)
    return np.array(image_list)

images = load_images(df_selected)

# 標籤編碼
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_selected['labels'])

# 分割數據集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 建立CNN模型
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(selected_breeds), activation='softmax')
])

# 編譯模型，使用带有自定义学习率的Adam优化器
learning_rate = 0.001
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# 訓練模型，使用数据增强
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=10,
          validation_data=(X_test, y_test))

# 評估模型
val_loss, val_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {val_acc}")
# accuracy = 0.31

Epoch 1/10
51/51 [==============================] - 262s 5s/step - loss: 2.8233 - accuracy: 0.0819 - val_loss: 2.6750 - val_accuracy: 0.0844
Epoch 2/10
51/51 [==============================] - 257s 5s/step - loss: 2.6771 - accuracy: 0.1018 - val_loss: 2.6631 - val_accuracy: 0.0918
Epoch 3/10
51/51 [==============================] - 263s 5s/step - loss: 2.5604 - accuracy: 0.1527 - val_loss: 2.3779 - val_accuracy: 0.1911
Epoch 4/10
51/51 [==============================] - 261s 5s/step - loss: 2.4513 - accuracy: 0.1670 - val_loss: 2.3520 - val_accuracy: 0.2060
Epoch 5/10
51/51 [==============================] - 263s 5s/step - loss: 2.3836 - accuracy: 0.2024 - val_loss: 2.2241 - val_accuracy: 0.2481
Epoch 6/10
51/51 [==============================] - 259s 5s/step - loss: 2.3199 - accuracy: 0.2259 - val_loss: 2.1777 - val_accuracy: 0.2605
Epoch 7/10
51/51 [==============================] - 256s 5s/step - loss: 2.2515 - accuracy: 0.2402 - val_loss: 2.2610 - val_accuracy: 0.2233
Epoch 8/10
51